In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
from gensim.models import Word2Vec
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from tqdm import tqdm
from nltk.corpus import stopwords
import pickle as pkl

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from sklearn.utils import shuffle
import string
import spacy
import re
import os
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin

from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

In [11]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 991.0 kB/s eta 0:00:13
      --------------------------------------- 0.3/12.8 MB 2.6 MB/s eta 0:00:05
     - -------------------------------------- 0.5/12.8 MB 3.3 MB/s eta 0:00:04
     -- ------------------------------------- 0.7/12.8 MB 3.7 MB/s eta 0:00:04
     -- ------------------------------------- 0.9/12.8 MB 3.9 MB/s eta 0:00:04
     --- ------------------------------------ 1.1/12.8 MB 4.0 MB/s eta 0:00:03
     ---- ----------------------------------- 1.4/12.8 MB 4.2 MB/s eta 0:00:03
     ----- ---------------------------------- 1.6/12.8 MB 4.3 MB/s eta 0:00:03
     ----- ---------------------------------- 1.8/12.8 MB 4.3 MB/s eta 0:00:03
     ------ --------------------------------- 2.1/12.8 MB 4.4 MB/s eta 0:00:03
     ------- -------------------------------- 2.3/12.8 MB 4.4 MB/s eta 0:00:03
     ------- -------------------------------- 2.4/12.8 MB 

In [12]:
import spacy

nlp = spacy.load('en_core_web_sm')
doc = nlp(sentence)

lemm_sentence = [token.lemma_ for token in doc]

' '.join(lemm_sentence)

NameError: name 'sentence' is not defined

In [6]:
ps = PorterStemmer()
puncts = string.punctuation
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
tags = re.compile(r'<.*?>')
remove_non_alpha = re.compile(r'\W')
spaces = re.compile(r'\s+')
not_alpha_numeric = re.compile(r'[^a-zA-Z0-9\s]')
urls = re.compile(r'http\S+|www\S+')
remove_non_ascii = re.compile(r'[^\x00-\x7F]+')
mentions = re.compile(r'[@#]\w+')

nlp = spacy.load('en_core_web_sm')

def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {
        'J' : wordnet.ADJ, 
        'N' : wordnet.NOUN,
        'V' : wordnet.VERB,
        'R' : wordnet.ADV
    }
    return tag_dict.get(tag, wordnet.NOUN)

def cleaning(text, stemming = True):
    text = text.lower()
    text = tags.sub('', text)
#     text = remove_notn_alpha.sub(, ' ', text) # we don't want to lose the numerical information
    text = spaces.sub(' ', text)
    text = not_alpha_numeric.sub(' ', text)
    text = urls.sub('', text)
#     text = re.sub(r'\d+', '', text)
    text = remove_non_ascii.sub('', text) # to remove non-ascii characters
    text = mentions.sub('', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    tokens = word_tokenize(text)
    
    # Stemming
    if stemming:
        s = [word for word in tokens if word not in stop_words]
        s = [ps.stem(word) for word in s]
        
    # Lemmatization
    else:
        doc = nlp(text)
        s = [token.lemma_ for token in doc]
        
    return ' '.join(s)

In [7]:
txt = 'Banking system low-cost deposits could decline further, says SBI chairman Setty SBI’s Casa ratio stood at 40.7% as on 30 June, down from 42.9% from the same period last year. Its loan book by a robust 15.4% year-on-year, outpacing deposit growth of 8.2% in the June quarter. Mumbai: The share of low-cost deposits in the banking system could decline further and go below the levels seen before covid-19 on the back of efficient cash management by the government, the chairman of India’s largest lender SBI C.S. Setty said.' 


sample = cleaning(txt, stemming=False)

In [36]:
sample

'banking system low cost deposit could decline far   say sbi chairman setty sbi s casa ratio stand at 40 7   as on 30 june   down from 42 9   from the same period last year   its loan book by a robust 15 4   year on year   outpace deposit growth of 8 2   in the june quarter   mumbai   the share of low cost deposit in the banking system could decline far and go below the level see before covid 19 on the back of efficient cash management by the government   the chairman of india s large lender sbi c s   setty say'

In [75]:
cleaned_data = data['article'].progress_apply(lambda x : cleaning(x, stemming = False))

100%|████████████████████████████████████████████████████████████████████████████| 12502/12502 [07:27<00:00, 27.92it/s]


In [2]:
with open('cleaned_data.pkl', 'rb') as file1:
    cleaned_data = pkl.load(file1)

In [3]:
with open('tokenized_data.pkl', 'rb') as file1:
    tokenized_data = pkl.load(file1)

In [70]:
data = pd.read_csv('dataset.csv')

In [71]:
data.dropna(inplace = True)

In [72]:
data.isnull().sum()

published_at       0
uuid               0
sentiment_score    0
article            0
dtype: int64

In [12]:
tqdm.pandas()
stop_words = set(stopwords.words('english'))

def tokenize(text):
    data = [word for word in word_tokenize(text) if word not in stop_words]
    return data

df = data['article'].progress_apply(tokenize)

100%|████████████████████████████████████████████████████████████████████████████| 12502/12502 [03:50<00:00, 54.19it/s]


In [14]:
def sent_to_vector(tokens, model):
    word_vectors = [model.wv.get_vector(word) for word in tokens if word in model.wv.key_to_index]
    
    if len(word_vectors) > 0:
        emb_vector = np.mean(word_vectors, axis = 0)
    else:
        emb_vector = np.zeros(model.vector_size)
    return emb_vector

In [67]:
cbow = Word2Vec(df, vector_size=20, workers=8, window = 5, sg = 1)

In [68]:
cbow_data = sent_to_vector(sample, cbow)

In [69]:
np.round(cbow_data, 2)

array([ 0.15, -0.31,  0.16,  0.22,  0.22, -0.7 ,  0.38,  0.17, -0.25,
        0.05,  0.27, -0.23, -0.01, -0.19, -0.21,  0.65,  0.67, -0.54,
       -0.26, -0.84], dtype=float32)

In [ ]:
[ 0.15, -0.31,  0.16,  0.22,  0.22, -0.7 ,  0.38,  0.17, -0.25, 0.05,  0.27, -0.23, -0.01, -0.19, -0.21,  0.65,  0.67, -0.54, -0.26, -0.84]

In [59]:
tf = TfidfVectorizer(sublinear_tf = True, ngram_range=(2, 2), max_features = 100, stop_words='english')
tf.fit(data)

TfidfVectorizer(max_features=100, ngram_range=(2, 2), stop_words='english',
                sublinear_tf=True)

In [60]:
tf.vocabulary_

{'sensex nifty': 81,
 'stock market': 86,
 'tata consultancy': 88,
 'consultancy service': 22,
 'state bank': 85,
 'bank india': 14,
 'ultratech cement': 93,
 'larsen toubro': 50,
 'adani port': 4,
 'bajaj finance': 7,
 'mahindra mahindra': 55,
 'reliance industry': 73,
 'bharti airtel': 17,
 'icici bank': 35,
 'hdfc bank': 32,
 'psu bank': 70,
 'bank index': 13,
 'tata motors': 90,
 'bajaj finserv': 8,
 'india sbi': 41,
 'punjab national': 72,
 'national bank': 61,
 'cent state': 20,
 'tata motor': 89,
 'bank hdfc': 11,
 'bse sensex': 18,
 'asian paint': 5,
 'hindustan unilever': 33,
 'india bank': 39,
 'rs 000': 76,
 '000 crore': 0,
 'fix deposit': 30,
 'senior citizen': 79,
 'axis bank': 6,
 'share price': 83,
 'crore state': 26,
 'include state': 36,
 'life insurance': 52,
 'union bank': 94,
 'lender state': 51,
 'bank state': 15,
 'indusind bank': 45,
 'public sector': 71,
 'reserve bank': 74,
 'country large': 24,
 'large lender': 49,
 'new delhi': 63,
 'basis point': 16,
 'bank 

In [61]:
np.round(tf.transform([sample]).toarray(), 2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.75, 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.66, 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  ]])

In [ ]:
[[0.  , 0.  , 0.  ,...... , 0.75, 0.  , 0.  ,... , 0.  , 0.66, 0.  , 0. , 0, ......]]

In [4]:
def load_glove_embeddings(file_path):
    embeddings_index = {}    
    with open(file_path, 'r', encoding = 'utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype = 'float32')
            embeddings_index[word] = coefs
    return embeddings_index

def create_embedding_matrix(tokenizer, embeddings_index, embedding_dim = 300):
    vocabulary_size = len(tokenizer.word_index) + 1
    embedding_matrix = np.zeros((vocabulary_size, embedding_dim))
    
    for word, i in tqdm(tokenizer.word_index.items(), desc = 'Creating Embeddings'):
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

def prepare_sequences(tokenizer, texts, max_length):
    sequences = tokenizer.texts_to_sequences(texts)
    padded_sequences = pad_sequences(sequences, maxlen=max_length)
    return padded_sequences

glove_file_path = 'D:/DATA (D)/Glove/glove.6B/glove.6B.300d.txt'
glove_embeddings = load_glove_embeddings(glove_file_path)

train_texts = cleaned_data

tokenize = Tokenizer()
tokenize.fit_on_texts(train_texts)

embedding_matrix = create_embedding_matrix(tokenize, glove_embeddings, embedding_dim = 300)

Creating Embeddings: 100%|███████████████████████████████████████████████████| 20396/20396 [00:00<00:00, 174200.74it/s]


In [18]:
max_length = 100
glove_data = prepare_sequences(tokenize, sample, max_length)

In [19]:
glove_data

array([[   0,    0,    0, ...,    0,    0, 1731],
       [   0,    0,    0, ...,    0,    0,   10],
       [   0,    0,    0, ...,    0,    0, 1506],
       ...,
       [   0,    0,    0, ...,    0,    0,   18],
       [   0,    0,    0, ...,    0,    0,   10],
       [   0,    0,    0, ...,    0,    0, 1062]])

In [8]:
import numpy as np

# Function to load GloVe embeddings
def load_glove_embeddings(glove_file_path):
    embeddings_index = {}
    with open(glove_file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = vector
    return embeddings_index

# Function to preprocess and get sentence embedding
def get_sentence_embedding(sentence, embeddings_index, embedding_dim=300):
    words = sentence.lower().split()  # Simple tokenization
    valid_embeddings = []
    
    for word in words:
        if word in embeddings_index:
            valid_embeddings.append(embeddings_index[word])
    
    if valid_embeddings:
        sentence_embedding = np.mean(valid_embeddings, axis=0)  # Averaging word embeddings
    else:
        sentence_embedding = np.zeros(embedding_dim)  # If no valid words, return zero vector

    return sentence_embedding

glove_file_path = 'D:/DATA (D)/Glove/glove.6B/glove.6B.300d.txt'
glove_embeddings = load_glove_embeddings(glove_file_path)

# Example usage
sentence = sample
embedding = get_sentence_embedding(sentence, glove_embeddings)
print(embedding)

[-1.35618508e-01  4.56813686e-02 -1.95589401e-02 -6.38279617e-02
 -3.28229629e-02  2.17241012e-02  2.29437966e-02  8.63058865e-02
 -3.18159871e-02 -1.44604933e+00  2.47517647e-03  9.91021022e-02
  6.75166165e-03  2.80945282e-02  6.51413351e-02  7.65789673e-02
 -5.98912202e-02 -3.99680063e-02 -5.86587116e-02 -1.73704758e-01
 -1.16417602e-01  3.62021364e-02  1.66413546e-01  8.97203237e-02
 -6.61192760e-02  3.10124122e-02  4.79388572e-02 -8.11925456e-02
 -1.65717408e-01 -3.12348418e-02 -1.58275999e-02  2.18933672e-01
 -7.41436705e-02  1.78346649e-01 -7.21798122e-01 -9.14353058e-02
  7.89156333e-02  7.15264454e-02 -4.06631045e-02 -1.42471604e-02
  1.24502657e-02 -9.78958085e-02 -1.71315238e-01  2.46180639e-01
  1.83217339e-02 -7.48973386e-03  2.11197920e-02  2.33086988e-01
 -1.72564805e-01 -3.23594734e-03  3.67420465e-02 -9.70273539e-02
 -4.87702601e-02 -2.65695956e-02  1.00972818e-03  2.94278488e-02
 -2.36095600e-02  1.33322984e-01  3.13311326e-03 -8.37082341e-02
  2.21162140e-02  2.10166

In [9]:
np.round(embedding, 2)

array([-0.14,  0.05, -0.02, -0.06, -0.03,  0.02,  0.02,  0.09, -0.03,
       -1.45,  0.  ,  0.1 ,  0.01,  0.03,  0.07,  0.08, -0.06, -0.04,
       -0.06, -0.17, -0.12,  0.04,  0.17,  0.09, -0.07,  0.03,  0.05,
       -0.08, -0.17, -0.03, -0.02,  0.22, -0.07,  0.18, -0.72, -0.09,
        0.08,  0.07, -0.04, -0.01,  0.01, -0.1 , -0.17,  0.25,  0.02,
       -0.01,  0.02,  0.23, -0.17, -0.  ,  0.04, -0.1 , -0.05, -0.03,
        0.  ,  0.03, -0.02,  0.13,  0.  , -0.08,  0.02,  0.02,  0.22,
       -0.2 ,  0.01, -0.34,  0.09, -0.02, -0.08, -0.02,  0.05,  0.26,
        0.08, -0.03, -0.1 , -0.  ,  0.1 , -0.07, -0.11, -0.01, -0.11,
       -0.11,  0.02,  0.09,  0.2 ,  0.1 , -0.15,  0.1 , -0.01,  0.09,
        0.03,  0.07, -0.24, -0.1 ,  0.03, -0.02, -0.31,  0.11,  0.03,
       -0.25,  0.03,  0.09,  0.02, -0.09, -0.11, -0.17,  0.08,  0.11,
       -0.12,  0.15, -0.06, -0.34, -0.12, -0.14,  0.07,  0.03,  0.07,
        0.08,  0.03, -0.25, -0.03, -0.14,  0.05,  0.12,  0.01,  0.02,
        0.02, -0.02,

In [ ]:
[-0.14,  0.05, -0.02, ......, -0.25,  0.03,  0.09,  0.02,...., 0.46, -0.02,-0.33, -0.1 , -0.01]

In [17]:
from scipy.spatial.distance import cosine

# Function to perform vector arithmetic
def vector_arithmetic(word1, word2, word3, embeddings_index):
    if word1 not in embeddings_index:
        print('not', word1)
    if word2 not in embeddings_index:
        print('not', word2)
    if word3 not in embeddings_index:
        print('not', word3)
    
#     if word1 not in embeddings_index or word2 not in embeddings_index or word3 not in embeddings_index:
#         print("One of the words is not in the vocabulary.")
        return None
    
    # Perform the vector arithmetic: word1 - word2 + word3
    result_vector = embeddings_index[word1] - embeddings_index[word2] + embeddings_index[word3]
    
    return result_vector

# Function to find the most similar word to a given vector
def find_closest_word(vector, embeddings_index):
    closest_word = None
    min_distance = float('inf')

    for word, embedding in embeddings_index.items():
        distance = cosine(vector, embedding)
        if distance < min_distance:
            min_distance = distance
            closest_word = word
    
    return closest_word

# Example usage
word1 = 'bankrupt'
word2 = 'loss'
word3 = 'benefit'

# Ensure GloVe embeddings are loaded
glove_embeddings = load_glove_embeddings(glove_file_path)

# Perform vector arithmetic: sbi - bank + investor
result_vector = vector_arithmetic(word1, word2, word3, glove_embeddings)

# Find the closest word to the resulting vector
if result_vector is not None:
    closest_word = find_closest_word(result_vector, glove_embeddings)
    print("Resulting word from 'sbi - bank + investor':", closest_word)

Resulting word from 'sbi - bank + investor': bankrupt


In [18]:
from scipy.spatial.distance import cosine
import heapq

# Function to perform vector arithmetic
def vector_arithmetic(word1, word2, word3, embeddings_index):
    if word1 not in embeddings_index or word2 not in embeddings_index or word3 not in embeddings_index:
        print("One of the words is not in the vocabulary.")
        return None
    
    # Perform the vector arithmetic: word1 - word2 + word3
    result_vector = embeddings_index[word1] - embeddings_index[word2] + embeddings_index[word3]
    
    return result_vector

# Function to find the closest n words to a given vector
def find_closest_words(vector, embeddings_index, n=3):
    word_distances = []
    
    # Calculate cosine distance for each word in embeddings_index
    for word, embedding in embeddings_index.items():
        distance = cosine(vector, embedding)
        word_distances.append((distance, word))
    
    # Get the n closest words by sorting by distance
    closest_words = heapq.nsmallest(n, word_distances, key=lambda x: x[0])
    
    return [word for _, word in closest_words]

# Example usage
word1 = 'bankrupt'
word2 = 'loss'
word3 = 'benefit'

# Load GloVe embeddings
glove_embeddings = load_glove_embeddings(glove_file_path)

# Perform vector arithmetic: sbi - bank + investor
result_vector = vector_arithmetic(word1, word2, word3, glove_embeddings)

# Find the closest 3 words to the resulting vector
if result_vector is not None:
    closest_words = find_closest_words(result_vector, glove_embeddings, n=3)
    print("Closest 3 words from 'sbi - bank + investor':", closest_words)

Closest 3 words from 'sbi - bank + investor': ['bankrupt', 'benefit', 'financially']
